In [1]:
import easyocr
import pandas as pd
import numpy as np
from os import listdir
from difflib import SequenceMatcher
from autocorrect import Speller

In [2]:
all_files =  ["monkey_puppet","surprised_pikachu","well_yes_but_actually_no","10_Guy","Spiderman_Computer_Desk", "Kevin_Hart", "laughing_leo", "Lisa", "Roll_Safe_Think_About_It", "Change_My_Mind", "Futurama_Fry", "First_World_Problems"]
reader = easyocr.Reader(['en']) 
spell = Speller(lang='en')

In [3]:
def only_text(lis):
    text = []
    for i in result:
        x = i[1]
        text.append(x)
    return text
def only_eval(lis):
    evl = []
    for i in result:
        x = float(i[2])
        evl.append(x)
    return evl
def purify(text):
    if not text:
        return "no text"
    pu_text = spell((text.lower()))
    waste = ["well yes but actually no", "change my mind"]
    for i in waste:
        if i in pu_text:
            pu_text.replace(i," ")
    sp_text = pu_text.split(" ")
    for i in range(0, len(sp_text)):
        if (SequenceMatcher(a=sp_text[i], b="imgflib").ratio() > .8) or (SequenceMatcher(a=sp_text[i], b="imgflib.com").ratio() > .8) or (sp_text[i] == "com"):
            del sp_text[i]
            break
    for i, t in enumerate(sp_text):
        if t.endswith(":") and ((sp_text[i-1]).lower() in "my her his him"):
            sp_text.insert(i-1,"\n")
        elif t.endswith(":"):
            sp_text.insert(i,"\n")
    return " ".join(sp_text)

In [4]:
text = []
all_class = []
for file in all_files:
    all_text = []
    cors_eval = []
    img = listdir(r"D:\{}".format(file))
    for i in range(len(img)):
        try:
            result = reader.readtext(r"D:\{}\{}".format(file, img[i]), paragraph=False)
            x = only_text(result)
            y = only_eval(result)
            all_text.append(x)
            cors_eval.append(y)
        except:
            pass
    for i, j in zip(all_text[:], cors_eval[:]):
        for t, e in zip(i[:], j[:]):
            if e <.5:
                i.remove(t)
                j.remove(e)
        if np.mean(j) < .7:
            all_text.remove(i)
            cors_eval.remove(j)
    joined_text = []
    for i in all_text:
        joined_text.append(" ".join(i))
    label = np.ones(len(all_text))*(all_files.index(file)+1)
    label-=1
    text.extend(joined_text)
    all_class.extend(label)

C:\Users\amrsh\anaconda3\lib\site-packages\numpy\core\fromnumeric.py:3372: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Users\amrsh\anaconda3\lib\site-packages\numpy\core\_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [ ]:
purified_text = []
for i in text:
    pu = purify(str(i))
    purified_text.append(pu)

In [ ]:
data = {"Text": purified_text, "Class": all_class}
df = pd.DataFrame(data)
df = df.drop_duplicates()
df.reset_index()
df.head(5)

In [ ]:
df.to_excel("NLP_classes.xlsx")

In [ ]:
df.value_counts(["Class"])